In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

model_data = pd.read_csv('data\model_data.csv')

Using TensorFlow backend.


In [2]:
model_data.dropna(inplace=True)

In [3]:
model_data.drop(['daily_percentage_change'], inplace=True, axis=1)

In [4]:
# 80% train and 20% test splitting

train = model_data[0:1965]
test = model_data[1965:2457]

In [5]:
scaler = MinMaxScaler()

x_train = train.drop(['close_future', 'date'], axis=1)
x_train = scaler.fit_transform(x_train)
y_train = train['close_future']
x_test = test.drop(['close_future', 'date'], axis=1)
x_test = scaler.fit_transform(x_test)
y_test = test['close_future']

In [6]:
y_train = np.array(y_train, dtype=float)
y_test = np.array(y_test, dtype=float)

In [7]:
# Reshaping to feed the model

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [8]:
y_test.shape

(492,)

In [9]:
x_train.shape

(1965, 20, 1)

In [10]:
y_train.shape

(1965,)

In [11]:
model = Sequential()  
model.add(LSTM(50, activation = 'relu', batch_input_shape=(None, 20, 1),return_sequences=False))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.summary()





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = model.fit(x_train, y_train, nb_epoch=50, verbose=2, validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1965 samples, validate on 492 samples
Epoch 1/50
 - 1s - loss: 153.2682 - val_loss: 50.8298
Epoch 2/50
 - 1s - loss: 61.8725 - val_loss: 13.1891
Epoch 3/50
 - 1s - loss: 11.2749 - val_loss: 24.3792
Epoch 4/50
 - 1s - loss: 1.6788 - val_loss: 26.9083
Epoch 5/50
 - 1s - loss: 1.0526 - val_loss: 29.8279
Epoch 6/50
 - 1s - loss: 0.7244 - val_loss: 28.7796
Epoch 7/50
 - 1s - loss: 0.6003 - val_loss: 32.2454
Epoch 8/50
 - 1s - loss: 0.5342 - val_loss: 31.9758
Epoch 9/50
 - 1s - loss: 0.5253 - val_loss: 31.4783
Epoch 10/50
 - 1s - loss: 0.5157 - val_loss: 35.1928
Epoch 11/50
 - 1s - loss: 0.4589 - val_loss: 33.6682
Epoch 12/50
 - 1s - loss: 0.5822 - val_loss: 33.8931
Epoch 13/50
 - 1s - loss: 0.4226 - val_loss: 33.4531
Epoch 14/50
 - 1s - loss: 0.4017 - val_loss: 35.4458
Epoch 15/50
 - 1s - loss: 0.4401 - val_loss: 35.7736
Epoch 16/50
 - 1s - loss: 0.4054 - val_loss: 28.5483
Epoch 17/50
 -

In [13]:
results = model.predict(x_test)
results_train = model.predict(x_train)

In [14]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=pd.to_datetime(train['date']), y= results_train[:,0], name='Modelo'))
fig.add_trace(go.Scatter(x=pd.to_datetime(train['date']), y=y_train, name='Real'))
fig.update_layout(title='<b> Valores preditos no treino </b>')
fig.update_yaxes(title_text='Preço do papel (R$)')
fig.show()

In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=pd.to_datetime(test['date']), y= results[:,0], name='Modelo'))
fig.add_trace(go.Scatter(x=pd.to_datetime(test['date']), y=y_test, name='Real'))
fig.update_layout(title='<b> Valores preditos no teste </b>')
fig.update_yaxes(title_text='Preço do papel (R$)')
fig.show()

In [16]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = np.array(range(400)), y=history.history['loss']))
fig.update_layout(title='<b> MSE ao longo do treino </b>')
fig.update_xaxes(title_text='Épocas')
fig.show()